In [5]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('linear-regression')
jovian.set_colab_id('1AKo4_TmdybnZAUVSmZJGvEX3Vov2WEqL')

     |████████████████████████████████| 71kB 3.3MB/s 


In [6]:
!pip install jovian --upgrade --quiet

In [7]:
import numpy as np
import torch

## Training data

We can represent the training data using two matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [8]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [9]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

Let's convert the arrays to PyTorch tensors.

In [10]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

In [11]:
# Weights and biases
w = torch.randn((2, 3), requires_grad=True)
b = torch.randn((2,), requires_grad=True)
print(w)
print(b)

tensor([[-0.7194,  0.6772,  0.8153],
        [-0.6113,  0.0631, -0.3842]], requires_grad=True)
tensor([-0.3664,  0.0724], requires_grad=True)


Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

# $$X \times w^T + b$$

We can define the model as follows:

In [12]:
def model(x):
  return torch.matmul(x, w.t()) + b

In [13]:
print(inputs)
print(w)
print(b)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[-0.7194,  0.6772,  0.8153],
        [-0.6113,  0.0631, -0.3842]], requires_grad=True)
tensor([-0.3664,  0.0724], requires_grad=True)


In [14]:
# Generate predictions
preds = model(inputs)
print(preds)
print('=' * 20)
print(targets)

tensor([[ 27.5471, -56.8465],
        [ 45.9397, -74.5935],
        [ 75.0756, -66.9428],
        [-14.4582, -73.7813],
        [ 72.0747, -62.9463]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to just work.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [15]:
# MSE loss
def mse(t1, t2):
  diff = t1-t2
  return torch.sum(diff * diff) / diff.numel()

In [16]:
loss = mse(preds, targets)
print(loss)

tensor(13853.0879, grad_fn=<DivBackward0>)


In [17]:
# diff = (preds - targets)**2
# loss = torch.mean(diff)
# loss

Here’s how we can interpret the result: On average, each element in the prediction differs from the actual target by the square root of the loss. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the loss because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [18]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [19]:
print(w)
print(w.grad)

tensor([[-0.7194,  0.6772,  0.8153],
        [-0.6113,  0.0631, -0.3842]], requires_grad=True)
tensor([[ -2988.3125,  -3082.8130,  -1905.7360],
        [-13297.5625, -14594.7422,  -9024.8418]])


In [20]:
import jovian

jovian

<module 'jovian' from '/usr/local/lib/python3.7/dist-packages/jovian/__init__.py'>

## Adjust weights and biases to reduce the loss
The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [21]:
print(w)
print(w.grad)

tensor([[-0.7194,  0.6772,  0.8153],
        [-0.6113,  0.0631, -0.3842]], requires_grad=True)
tensor([[ -2988.3125,  -3082.8130,  -1905.7360],
        [-13297.5625, -14594.7422,  -9024.8418]])


In [22]:
# Let's take learning rate lr = 1e-5
lr = 1e-5
with torch.no_grad():
  w -= w.grad * lr
  b -= b.grad * lr

In [23]:
print(w)
print(b)

tensor([[-0.6895,  0.7080,  0.8343],
        [-0.4783,  0.2090, -0.2940]], requires_grad=True)
tensor([-0.3661,  0.0740], requires_grad=True)


We multiply the gradients with a very small number ($10^{-5}$ in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.  

Before we proceed to verify that the loss is actually lower, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [24]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [25]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 32.6138, -33.4785],
        [ 52.5920, -43.8718],
        [ 82.9120, -30.5809],
        [ -9.3790, -50.6013],
        [ 78.4305, -33.4410]], grad_fn=<AddBackward0>)


In [26]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(9361.1191, grad_fn=<DivBackward0>)


In [27]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -2465.5869,  -2522.1084,  -1559.5426],
        [-10884.5146, -12000.5635,  -7424.2373]])
tensor([ -28.7661, -130.3947])


Let's update the weights and biases using the gradients computed above.

In [28]:
# Adjust weights & reset gradients
with torch.no_grad():
  w -= w.grad * lr
  b -= b.grad * lr
  w.grad.zero_()
  b.grad.zero_()

Let's take a look at the new weights and biases.

In [29]:
print(w)
print(b)

tensor([[-0.6648,  0.7332,  0.8499],
        [-0.3694,  0.3290, -0.2197]], requires_grad=True)
tensor([-0.3658,  0.0753], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [30]:
# Calculate loss
print(f"loss before: {loss}")
preds = model(inputs)
loss = mse(preds, targets)
print(f"loss after: {loss}")

loss before: 9361.119140625
loss after: 6333.85009765625


## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [31]:
for i in range(100):
  # generate predictions
  preds = model(inputs)
  # calculate the loss
  loss = mse(preds, targets)
  # compute gradients
  loss.backward()
  # adjust the weights and reset the gradients
  with torch.no_grad():
      w -= w.grad * lr
      b -= b.grad * lr
      w.grad.zero_()
      b.grad.zero_()


Once again, let's verify that the loss is now lower:

In [32]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(f"loss= {loss}")

loss= 42.51047897338867


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [33]:
# predictions
print(preds)

print("=" * 50)

# targets
print(targets)


tensor([[ 56.3440,  72.3712],
        [ 83.3182,  96.5108],
        [117.4845, 139.1031],
        [ 16.8314,  48.3633],
        [106.2621, 105.2955]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs. 

In [34]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/boubekri/linear-regression


'https://jovian.ai/boubekri/linear-regression'

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [35]:
import torch.nn as nn
import numpy as np

As before, we represent the inputs and targets and matrices.

In [36]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We are using 15 training examples to illustrate how to work with large datasets in small batches. 

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [37]:
from torch.utils.data import TensorDataset

In [38]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [39]:
from torch.utils.data import DataLoader

In [40]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop. Let's look at an example.

In [41]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[103.,  43.,  36.],
        [102.,  43.,  37.],
        [101.,  44.,  37.],
        [ 68.,  97.,  70.],
        [ 92.,  87.,  64.]])
tensor([[ 20.,  38.],
        [ 22.,  37.],
        [ 21.,  38.],
        [102., 120.],
        [ 82., 100.]])


In each iteration, the data loader returns one batch of data with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [42]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1575,  0.3732, -0.1434],
        [-0.1872,  0.2635,  0.3903]], requires_grad=True)
Parameter containing:
tensor([-0.3389, -0.3745], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [43]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.1575,  0.3732, -0.1434],
         [-0.1872,  0.2635,  0.3903]], requires_grad=True),
 Parameter containing:
 tensor([-0.3389, -0.3745], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [44]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 7.0002, 20.3964],
        [ 8.9910, 30.7560],
        [27.6472, 41.2863],
        [-5.6642,  6.3003],
        [14.5819, 39.3248],
        [ 6.4695, 19.9456],
        [ 8.4745, 30.8828],
        [27.3463, 41.4893],
        [-5.1335,  6.7511],
        [14.5960, 39.9023],
        [ 6.4837, 20.5231],
        [ 8.4603, 30.3053],
        [28.1637, 41.1596],
        [-5.6783,  5.7229],
        [15.1126, 39.7756]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [45]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [46]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [47]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(4748.3506, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [48]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

*Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.*

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [49]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [50]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 95.8001
Epoch [20/100], Loss: 151.8386
Epoch [30/100], Loss: 145.7127
Epoch [40/100], Loss: 39.6453
Epoch [50/100], Loss: 25.2338
Epoch [60/100], Loss: 39.2965
Epoch [70/100], Loss: 38.4431
Epoch [80/100], Loss: 17.3830
Epoch [90/100], Loss: 26.4643
Epoch [100/100], Loss: 13.8325


Let's generate predictions using our model and verify that they're close to our targets.

In [52]:
# Generate predictions
preds = model(inputs)
print(preds)

print("=" *50)

# Compare with targets
targets

tensor([[ 57.8156,  70.9983],
        [ 79.1571,  99.5182],
        [122.8444, 134.0571],
        [ 26.2184,  41.5481],
        [ 93.6764, 114.4449],
        [ 56.6136,  69.9843],
        [ 78.4531,  99.3881],
        [122.8638, 134.5632],
        [ 27.4204,  42.5621],
        [ 94.1744, 115.3288],
        [ 57.1117,  70.8682],
        [ 77.9551,  98.5042],
        [123.5483, 134.1872],
        [ 25.7203,  40.6642],
        [ 94.8783, 115.4589]], grad_fn=<AddmmBackward>)


tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [53]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.7450, 68.0152]], grad_fn=<AddmmBackward>)

In [54]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/boubekri/linear-regression


'https://jovian.ai/boubekri/linear-regression'